In [1]:
from IPython.display import clear_output
from functions import *
import numpy as np
import pandas as pd
from transformers import *
import tensorflow as tf
import pickle
import scipy as sc
import math as mt
# from scipy import cluster as clst
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.decomposition import PCA

## Import data

In [24]:
#dfd = pd.read_csv('sts-dev.csv', delimiter='\t' , error_bad_lines=False)
#STS-B
#dfd = pd.read_csv('sts-test.csv', delimiter='\t' , error_bad_lines=False)
#STS-2012
#dfd = pd.read_csv('../data/sts-2012-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2013
#dfd = pd.read_csv('../data/sts-2013-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2014
#dfd = pd.read_csv('../data/sts-2014-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2015
# dfd = pd.read_csv('../data/sts-2015-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
# dfd = dfd[dfd["corr"] != -1]
#STS-2016
# dfd = pd.read_csv('../data/sts-2016-test/STS2016.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
# dfd = dfd[dfd["corr"] != -1]
#SICK-R
dfd = pd.read_csv('../data/sick-r-test/SICK.csv', delimiter='\t' , error_bad_lines=False)
clear_output()

In [25]:
dfd

,pair_ID,sentence1,sentence2,entailment_label,corr,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,SemEval_set
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN
3,4,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRIAL
4,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...
9835,9996,A man is in a parking lot and is playing tenni...,The snowboarder is leaping fearlessly over whi...,NEUTRAL,1.0,A_neutral_B,B_neutral_A,A man is playing tennis with himself against a...,A snowboarder launches into the air over white...,FLICKR,FLICKR,TEST
9836,9997,Someone is boiling okra in a pot,The man is not playing the drums,NEUTRAL,1.0,A_neutral_B,B_neutral_A,someone is boiling okra in a pot,the man is playing the drums,SEMEVAL,SEMEVAL,TRAIN
9837,9998,The man is singing heartily and playing the gu...,A bicyclist is holding a bike over his head in...,NEUTRAL,1.0,A_neutral_B,B_neutral_A,the man sang and played his guitar,a bicyclist holds their bike over their head b...,SEMEVAL,FLICKR,TRAIN
9838,9999,A man in blue has a yellow ball in the mitt,A man is jumping rope outside,NEUTRAL,1.2,A_neutral_B,B_neutral_A,a man is jumping rope outside,a woman in blue has a yellow ball in her mitt,SEMEVAL,FLICKR,TRAIN


In [27]:
# Check for errors
dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
len(dfd)

9840

## Loading models - choose and run one cell

In [15]:
# BERT
casing = "bert-base-uncased" 
tokenizer = BertTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)

config = BertConfig(hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2 )
config.output_hidden_states = True

model = TFBertModel.from_pretrained(casing, config = config)
model.trainable = False
emb_len = 768
clear_output()

# BERT
n_cluster = 27 # Number of clusters to use
n_pc = 12 # Number of main principal components to drop for local method
n_pc_global = 15 # Number of main principal components to drop for global method

In [2]:
# GPT-2
casing = "gpt2" 
tokenizer = GPT2Tokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = GPT2Config()
config.output_hidden_states = True

model = TFGPT2Model.from_pretrained(casing, config=config)
model.trainable = False

emb_len = 768
clear_output()

# GPT2
n_cluster = 10
n_pc = 30
n_pc_global = 30

In [4]:
# RoBERTa
casing = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = RobertaConfig.from_pretrained(casing)
config.output_hidden_states = True

model = TFRobertaModel.from_pretrained(casing, config=config)
model.trainable = False
emb_len = 768
clear_output()

# RoBERTa
n_cluster = 27
n_pc = 12
n_pc_global = 25

## Get sentences and words representations

In [43]:
# Get sentence representations BERT/GPT2/RoBERTa
#sentences = get_representations(dfd, tokenizer, model, emb_len)
sentences = np.load("roberta-sick-test.npy", allow_pickle=True).tolist()
words = getWords(sentences)
clear_output()

In [33]:
# HELPER
np.save("gpt2-sick-test.npy", np.asarray(sentences, dtype=object))

### Run multiple experiments...

In [7]:
# datasets = [ "sts-test.csv", "../data/sts-2015-test/STS.input.ALL.csv", "../data/sts-2014-test/STS.input.ALL.csv", "../data/sts-2013-test/STS.input.ALL.csv", "../data/sts-2012-test/STS.input.ALL.csv"]
# npynames = ["gpt2-stsb-test.npy", "gpt2-sts2015-test.npy", "gpt2-sts2014-test.npy", "gpt2-sts2013-test.npy", "gpt2-sts2012-test.npy" ]

# for i in range(len(datasets)):
#     dsName = datasets[i]
#     npyName = npynames[i]
#     dfd = pd.read_csv(dsName, delimiter='\t' , error_bad_lines=False)
#     dfd = dfd[dfd["corr"] != -1]
#     dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
#     sentences = get_representations(dfd, tokenizer, model, emb_len)
#     np.save(npyName, np.asarray(sentences, dtype=object))


## Get representations via baseline, global method, local (isotropy enhancing) method 

In [ ]:
# BASELINE
baseline_sentence_rep = mean_pooling(words, sentences)
baseline_score = similarity(baseline_sentence_rep, emb_len)

# GLOBAL METHOD
global_representations = global_method(np.asarray(words), n_pc_global, emb_len)
global_sentence_rep = mean_pooling(global_representations, sentences)
global_score = similarity(global_sentence_rep, emb_len)

# LOCAL METHOD
isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
# calculating sentence representations based on mean word (isotropised) representations
sentence_rep = mean_pooling(isotropic_representations, sentences)
local_score = similarity(sentence_rep, emb_len)

In [47]:
# performance
print("Spearman Correlation baseline: ",sper_corrcoef(dfd['corr'], baseline_score))
print("Spearman Correlation global: ",sper_corrcoef(dfd['corr'], global_score))
print("Spearman Correlation local: ",sper_corrcoef(dfd['corr'], local_score))

# isotropy of space
print("Isotropy baseline: ", isotropy(np.asarray(baseline_sentence_rep, dtype=np.float64)))
print("Isotropy global: ", isotropy(global_representations))
print("Isotropy local: ", isotropy(isotropic_representations))

Spearman Correlation baseline:  62.38745752100364
Spearman Correlation global:  65.82563567035892
Spearman Correlation local:  64.72507185296475
Isotropy baseline:  5.857797332824382e-06
Isotropy global:  0.9053698891016388
Isotropy local:  0.9506556580464818


In [48]:
# REPEAT LAST METHOD
isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
sentence_rep = mean_pooling(isotropic_representations, sentences)
local_score = similarity(sentence_rep, emb_len)

print("Spearman Correlation local: ",sper_corrcoef(dfd['corr'], local_score))
#print("Isotropy local: ", isotropy(isotropic_representations))

Spearman Correlation local:  65.07141874721928


### Run multiple experiments...

In [5]:
# npynames = ["roberta-stsb-test.npy", "roberta-sts2015-test.npy", "roberta-sts2014-test.npy", "roberta-sts2013-test.npy", "roberta-sts2012-test.npy" ]
# datasets = [ "sts-test.csv", "../data/sts-2015-test/STS.input.ALL.csv", "../data/sts-2014-test/STS.input.ALL.csv", "../data/sts-2013-test/STS.input.ALL.csv", "../data/sts-2012-test/STS.input.ALL.csv"]
# appendfile = open("results3.txt", "a")

# for i in range(len(npynames)):
#     dsName = datasets[i]
#     dfd = pd.read_csv(dsName, delimiter='\t' , error_bad_lines=False)
#     dfd = dfd[dfd["corr"] != -1]
#     dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
#     sentences = np.load(npynames[i], allow_pickle=True).tolist()
#     words = getWords(sentences)

#     baseline_sentence_rep = mean_pooling(words, sentences)
#     baseline_score = similarity(baseline_sentence_rep, emb_len)

#     # GLOBAL METHOD
#     global_representations = global_method(np.asarray(words), n_pc_global, emb_len)
#     global_sentence_rep = mean_pooling(global_representations, sentences)
#     global_score = similarity(global_sentence_rep, emb_len)

#     # LOCAL METHOD
#     isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
#     # calculating sentence representations based on mean word (isotropised) representations
#     sentence_rep = mean_pooling(isotropic_representations, sentences)
#     local_score = similarity(sentence_rep, emb_len)

#     appendfile.write(npynames[i]+"\n")
#     appendfile.write("Spearman Correlation baseline: " + str(sper_corrcoef(dfd['corr'], baseline_score)) + "\n")
#     appendfile.write("Spearman Correlation global: " + str(sper_corrcoef(dfd['corr'], global_score))+ "\n")
#     appendfile.write("Spearman Correlation local: " + str(sper_corrcoef(dfd['corr'], local_score))+ "\n")
#     appendfile.flush()
#     # isotropy of space
#     appendfile.write("Isotropy baseline: " + str(isotropy(np.asarray(baseline_sentence_rep, dtype=np.float64)))+ "\n")
#     appendfile.write("Isotropy global: " + str(isotropy(global_representations))+ "\n")
#     appendfile.write("Isotropy local: " + str(isotropy(isotropic_representations))+ "\n")
#     appendfile.write("---------\n")
#     appendfile.flush()